##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# SavedModel Migration

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/migrate/saved_model">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/migrate/saved_model.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/migrate/saved_model.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/migrate/saved_model.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Once you have migrated your model from graphs and sessions to `tf.function`, `tf.Module`, and `tf.keras.Model`, it is time to migrate the saving and loading code. This is specifically a guide about migrating from TF1 to TF2, [see here a more general guide about the TF2 SavedModel API](../../guide/saved_model.ipynb).

This is a quick overview of the API changes from TF1 to TF2.

||TF1|Migration to TF2|
| ----| ----|---|
|**Saving**|`tf.compat.v1.saved_model.Builder`<br>`tf.compat.v1.saved_model.simple_save`|`tf.saved_model.save`<br>`tf.keras.models.save_model`
|**Loading**|`tf.compat.v1.saved_model.load`|`tf.saved_model.load`
|**Signatures**: a set of input <br>and output tensors that <br>can be used to run the<br> model|These are generated using the `*.signature_def` utils<br>(e.g. `tf.compat.v1.saved_model.predict_signature_def`)|Write a `tf.function`, and export it using the<br> `signatures` argument in <br>`tf.saved_model.save`.
**Classify and regress**:<br>special types of signatures|Generated with `classification_signature_def`, <br> `regresssion_signature_def`, and certain Estimator exports.|These two signatures types have been removed <br>from TF2.If the serving library requires these <br>method names, use the [`MethodNameUpdater`](https://www.tensorflow.org/api_docs/python/tf/compat/v1/saved_model/signature_def_utils/MethodNameUpdater).

For a more in-depth explanation of the mapping see the *Changes from TF1 to TF2* section

Below are code examples of writing saving and loading code in TF1 and TF2.

## Code Sample Set up

The code examples below show how to export and load the same dummy TensorFlow model (`add_two`) to SavedModel using the TF1 and TF2 APIs.

Run the below code to set up the imports and utils functions

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1
import shutil

def remove_dir(path):
  try:
    shutil.rmtree(path)
  except:
    pass
    
def add_two(input):
  return input + 2

## Building TF1 SavedModels


The TF1 APIs, `tf.compat.v1.saved_model.Builder`, `tf.compat.v1.saved_model.simple_save`, and `tf.estimator.Estimator.export_saved_model` export the TensorFlow graph and session.

### Builder


In [ ]:
remove_dir("saved-model-builder")

with tf.Graph().as_default() as g:
  with tf1.Session() as sess:
    input = tf1.placeholder(tf.float32, shape=[])
    output = add_two(input)
    print("add two output: ", sess.run(output, {input: 3.}))

    # Save with SavedModelBuilder
    builder = tf1.saved_model.Builder('saved-model-builder')
    sig_def = tf1.saved_model.predict_signature_def(
        inputs={'input': input}, 
        outputs={'output': output})
    builder.add_meta_graph_and_variables(
        sess, tags=["serve"], signature_def_map={
            tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY: sig_def
    })
    builder.save()

In [ ]:
!saved_model_cli run --dir simple-save --tag_set serve \
 --signature_def serving_default --input_exprs input=10

### Simple Save

In [ ]:
remove_dir("simple-save")

with tf.Graph().as_default() as g:
  with tf1.Session() as sess:
    input = tf1.placeholder(tf.float32, shape=[])
    output = add_two(input)
    print("add two output: ", sess.run(output, {input: 3.}))

    tf1.saved_model.simple_save(
        sess, 'simple-save',
        inputs={'input': input}, 
        outputs={'output': output})

In [ ]:
!saved_model_cli run --dir simple-save --tag_set serve \
 --signature_def serving_default --input_exprs input=10

### Estimator export

In the definition of the Estimator `model_fn`, you can define signatures in your model by returning `export_outputs` in the `EstimatorSpec`. There are different types of outputs:
* `tf.estimator.export.ClassificationOutput`
* `tf.estimator.export.RegressionOutput`
* `tf.estimator.export.PredictOutput`

These will produce `classify`, `regress`, and `predict` signature types. 

When the estimator is exported with `tf.estimator.Estimator.export_saved_model`, these signatures will be saved with the model.

In [ ]:
def model_fn(features, labels, mode):
  output = add_two(features['input'])
  step = tf1.train.get_global_step()
  return tf.estimator.EstimatorSpec(
      mode,
      predictions=output,
      train_op=step.assign_add(1),
      loss=tf.constant(0.),
      export_outputs={
          tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY: \
          tf.estimator.export.PredictOutput({'output': output})})
est = tf.estimator.Estimator(model_fn, 'estimator-checkpoints')

# Train for one step to create a checkpoint
def train_fn():
  return tf.data.Dataset.from_tensors({'input': 3.})
est.train(train_fn, steps=1)

# This util function `build_raw_serving...` takes in raw tensor features
# and builds an "input serving receiver function", which creates placeholder
# inputs to the model.
serving_input_fn = tf.estimator.export.build_raw_serving_input_receiver_fn(
    {'input': tf.constant(3.)})  # Pass in a dummy input batch
estimator_path = est.export_saved_model('exported-estimator', serving_input_fn)

# Estimator's export_saved_model creates a timestamped directory. Move this
# to a set path so it can be inspected with saved_model_cli in the cell below:
!rm -rf estimator-model
import shutil
shutil.move(estimator_path, 'estimator-model')

In [ ]:
!saved_model_cli run --dir estimator-model --tag_set serve \
 --signature_def serving_default --input_exprs input=[10]

## Building TF2 SavedModels

### TF2 Saving

To export your model in TF2, you must define a `tf.Module` or `tf.keras.Model` to hold all of your model's variables and functions. Then, call `tf.saved_model.save` to create a SavedModel.

In [ ]:
class MyModel(tf.Module):
  @tf.function
  def __call__(self, input):
    return add_two(input)

model = MyModel()

@tf.function
def serving_default(input):
  return {'output': model(input)}

signature_function = serving_default.get_concrete_function(
    tf.TensorSpec(shape=[], dtype=tf.float32))
tf.saved_model.save(
    model, 'tf2-save', signatures={
        tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature_function})

In [ ]:
!saved_model_cli run --dir tf2-save --tag_set serve \
 --signature_def serving_default --input_exprs input=10

### Keras SavedModel (TF2)

The Keras `tf` format exports a SavedModel from a `tf.keras.Model`. SavedModels exported with Keras can be reloaded using any of the loading APIs, in addition to the Keras loading function.

In [ ]:
inp = tf.keras.Input(3)
out = add_two(inp)
model = tf.keras.Model(inputs=inp, outputs=out)

@tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.float32)])
def serving_default(input):
  return {'output': model(input)}

model.save('keras-model', save_format='tf', signatures={
        tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY: serving_default})

In [ ]:
!saved_model_cli run --dir keras-model --tag_set serve \
 --signature_def serving_default --input_exprs input=10

## Loading SavedModels

SavedModels saved with any of the above APIs can be loaded with either TF1 or TF2. 

TF1 SavedModel can generally be used for inference when loaded into TF2, but training (generating gradients) is only possible if the SavedModel contains *resource variables*. You can check the dtype of the variables. If the variable dtype contains "_ref" then it is a reference variable.

TF2 SavedModel can be loaded and executed from TF1 as long as the SavedModel is saved with signatures.

The sections below contain code samples showing how to load the SavedModels saved in the previous sections, and call the exported signature.

### TF1 Loading

TF1 imports the SavedModel directly in to the current graph and session. You can call `session.run` on the tensor input and output names.

In [ ]:
def load_tf1(path, input):
  print('Loading from', path)
  with tf.Graph().as_default() as g:
    with tf1.Session() as sess:
      meta_graph = tf1.saved_model.load(sess, ["serve"], path)
      sig_def = meta_graph.signature_def[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
      input_name = sig_def.inputs['input'].name
      output_name = sig_def.outputs['output'].name
      print('  Output with input', input, ': ', 
            sess.run(output_name, feed_dict={input_name: input}))

load_tf1('saved-model-builder', 5.)
load_tf1('simple-save', 5.)
load_tf1('estimator-model', [5.])  # Estimator's input must be batched.
load_tf1('tf2-save', 5.)
load_tf1('keras-model', 5.)

### TF2 Loading

In TF2, objects are loaded into a Python object that stores the variables and functions. This is compatible with models saved from TF1. See `tf.saved_model.load` for details.

In [ ]:
def load_tf2(path, input):
  print('Loading from', path)
  loaded = tf.saved_model.load(path)
  out = loaded.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY](
      tf.constant(input))['output']
  print('  Output with input', input, ': ', out)

load_tf2('saved-model-builder', 5.)
load_tf2('simple-save', 5.)
load_tf2('estimator-model', [5.])  # Estimator's input must be batched.
load_tf2('tf2-save', 5.)
load_tf2('keras-model', 5.)

Models saved with the TF2 API can also access tf.functions and variables that are attached to the model (instead of those exported as signatures). Example:

In [ ]:
loaded = tf.saved_model.load('tf2-save')
print('restored __call__:', loaded.__call__)
print('output with input 5.', loaded(5))

### Keras loading

The Keras loading API allows you to reload a saved model back into a Keras Model object. Note that this only allows you to load SavedModels saved with Keras (`model.save` or `tf.keras.models.save_model`). Models saved with `tf.saved_model.save` can not be loaded.

Load the previously saved Keras model:

In [ ]:
loaded_model = tf.keras.models.load_model('keras-model')
loaded_model.predict_on_batch(tf.constant([1, 3, 4]))

## Changes from TF1 to TF2
This section lists out key terms from TF1, their TF2 equivalents, and what has changed.

### SavedModel

SavedModel is a format that stores a TensorFlow model. It contains signatures which are used by serving platforms to run the model.

The file format itself has not changed significantly, so SavedModels can be loaded and served using either TF1 and TF2 APIs. 

**Differences between TF1 and TF2**

The *serving* and *inference* use cases have not been updated, aside from API changes. The improvement in TF2 is the ability to *reuse* and *compose models* loaded from SavedModel.

In TF2, the program is represented by objects like `tf.Variable`, `tf.Module`, or Keras models and layers. There are no more global variables that have values stored in a session, and the graph now exists in different `tf.functions`. Consequently, during export, SavedModel saves each component and function graphs separately.

When you write a TensorFlow program with the TF2 Python API, you must build an object to manage the variables, functions, and other resources. Generally, this is accomplished by using the Keras API, but you can also build the object by creating or subclassing `tf.Module`.

Keras models and `tf.Module` automatically track variables and functions attached to them. SavedModel saves these connections between modules, variables and functions so that they can be restored when loading.


### Signatures

Signatures are the endpoints of a SavedModel -- they tell the user how to run the model and what inputs are needed. 

In TF1, signatures are created by listing the input and output tensors. In TF2, signatures are generated by passing in *concrete functions*.

To read more about TensorFlow functions, [see this guide](../guide/intro_to_graphs). In short, a concrete function is generated from a `tf.function`:
```
# Option 1: specify an input signature
@tf.function(input_signature=[...])
def fn(...):
  ... 
  return outputs

tf.saved_model.save(model, path, signatures={
    'name': fn
})
```
```
# Option 2: call get_concrete_function
@tf.function
def fn(...):
  ...
  return outputs

tf.saved_model.save(model, path, signatures={
    'name': fn.get_concrete_function(...)
})
```

### `session.run`

In TF1, you could call `session.run` with the imported graph as long as you already know the tensor names. This allows you to retrieve the restored variable values, or run parts of the model that were not exported in the signatures.

In TF2, you can directly access the variable (e.g. `loaded.dense_layer.kernel`), or call `tf.function`s attached the model object (e.g. `loaded.__call__`).

Unlike TF1, there is no way to extract parts of a function and access intermediate values. You *must* export all of the needed functionality in the saved object.


## Serving Migration notes

SavedModel was originally created to work with [TensorFlow Serving](https://www.tensorflow.org/tfx/guide/serving). This platform offers different types of prediction requests: classify, regress, and predict.

The TF1 API allows you to create these types of signatures with the utils: 
* `tf.compat.v1.saved_model.classification_signature_def`
* `tf.compat.v1.saved_model.regression_signature_def`
* `tf.compat.v1.saved_model.predict_signature_def`

Classify and regress restrict the inputs and outputs, so the inputs must be a `tf.Example`, and the outputs must be `classes` or `scores` or `prediction`. Meanwhile, the predict signature has no restrictions.

SavedModels exported with the **TF2** API are compatible with TensorFlow serving, but will only contain `predict` signatures. The `classify` and `regress` signatures have ben removed. 

If you require the use of the `classify` and `regress` signatures, you may modify the exported SavedModel using `tf.compat.v1.saved_model.signature_def_utils.MethodNameUpdater`.

## Other Saving Migration guides

If you are using TF Hub, then here are some guides that you may fine useful:
* https://www.tensorflow.org/hub/model_compatibility
* https://www.tensorflow.org/hub/migration_tf2
